In [1]:
from IPython.display import Latex

# TP1 : Parcours des protons dans la matière

## 1.

   [METTRE RÉFÉRENCE] nous informe que l'énergie cinétique des protons utilisés en protonthérapie varie entre 11 et 220 MeV, énergies correspondant à des distances de pénétration respectives de 1mm et 30 cm. Cette dernière étant la plus creuse que nous pouvons atteindre, on en déduit que l'énergie maximale typiquement utilisée est 220 MeV. 

### Relation entre le facteur de Lorentz $\gamma$ et l'énergie du proton $T$
l'énergie totale d'un proton est la somme de son énergie de masse au repos et de son énergie cinétique tel que,

\begin{align}
E = m_p c^2 + T
\end{align}

Par contre, l'énergie totale d'un proton en mouvement est aussi éagale à $\gamma m_p c^2$. On peut ainsi écrire, 

\begin{align}
E = \gamma m_p c^2 &= m_p c^2 + T\\
\Rightarrow\ T &=  m_p c^2 (\gamma - 1)
\end{align}

Bref, en considérant la masse du proton en électron-Volts, on peut réécrire l'énergie cinétique $T$ comme,

\begin{align}
T =  m_p (\gamma - 1)\quad [eV]
\end{align}

 
### Relation entre le facteur de Lorentz $\gamma$ et la vitesse réduite $\beta$  

Sachant que $\gamma$ et $\beta$ sont définit comme,

\begin{align}
\gamma = \frac{1}{\sqrt{1-v^2/c^2}}, \qquad \beta = \frac{v}{c}
\end{align}

leur relation s'exprime comme

\begin{align}
\Rightarrow\ \gamma = \frac{1}{\sqrt{1-\beta^2}}, \qquad \beta = \sqrt{\frac{\gamma^2 - 1}{\gamma^2}}
\end{align}

## 2. 


In [4]:
import numpy as np
import matplotlib.pyplot as plt


##5
En observant l'expression de la portée des protons, on remarque qu'elle n'est pas intégrable analytiquement, au moins sans faire d'énormes approximations. En effet, elle contient un log d'un polynome dépendant de l'énergie cinétique. La solution est donc simplement d'évaluer l'intégrale numériquement sur des bornes définies.

##6
On entre simplement la formule désirée dans la code si dessous avec les bons noms de constantes et il calculera l'intégrale définie en fonction de l'énergie cinétique en utilisant la méthode des trapèses. L'espression est laissée en sympy jusqu'au dernier moment possible pour permettre la réutilisation du code pour le numéro suivant.

In [ ]:
import sympy
from sympy import lambdify
import numpy as np

#symboles sympy
re, me, c , ne, bet2, gam, I, rho, mp, T = sympy.symbols("r_e m_e c n_e \\beta^2 \gamma I_x \\rho m_p T")

#constantes
rayon_electron  = 2.8179*10**(-15)*100 #cm
masse_electron  = 0.511                #Mev
masse_proton    = 938                  #Mev
vitesse_lumiere = 299792458*100        #cm/s
nombre_avogadro = 6.0221409*10**23     #1/mol

#fonctions
facteur_de_lorenz_gamma = lambda T : T/mp + 1
facteur_de_lorenz_beta_carre = lambda T:  (1- gam**(-2))
Temax = (2*me*(gam**2-1))/(1 + 2*gam*me/mp + me**2/mp**2)
Scol =  2*np.pi*re**2*me*ne/bet2 *(sympy.log(2*me*bet2*gam**2*Temax/I**2) - 2*bet2)
derivRSDCA =  rho/Scol

#fonction qui entre les constantes dans l'équation puis simplifie la substitutions des autres variables
derivRSDCA_subs = lambda densite, densite_electronique, excitation : (derivRSDCA
                .subs(bet2, facteur_de_lorenz_beta_carre(T))
                .subs(gam, facteur_de_lorenz_gamma(T))
                .subs(re, rayon_electron)
                .subs(me, masse_electron)
                .subs(ne, densite_electronique)
                .subs(mp, masse_proton)
                .subs(I, excitation)
                .subs(rho, densite))

#méthode des trapezes générale
def integral_trapese(fonction, resolution, debut, fin):
    trapeze = lambda values, interval: interval * (values[0] / 2 + np.sum(values[1:-1]) + values[-1] / 2)
    nombre = 1
    output_change = 1337
    while abs(output_change) > resolution :
        interval = (fin - debut) / nombre
        xvalues = [i for i in np.arange(debut, fin + interval, interval)]
        yvalues = [fonction(i) for i in xvalues]
        integral_result = trapeze(yvalues, interval)
        if nombre != 1:
            output_change = integral_result - prev
        prev = integral_result
        nombre = nombre * 2

        #pourcentage de complétude
        print("\r{}%".format(round(((abs(resolution)/abs(interval))*100),1)), end = "")

    print("\r",end="")
    return integral_result, nombre//2

#substitutions des constantes dépendantes du matériau dans l'équation
derivRSDCA_eau = lambda energie_cinetique : derivRSDCA_subs(
    densite              = 1.85,            #g/cm**3
    excitation           = 75*10**(-6),     #MeV
    densite_electronique = 5/9*nombre_avogadro).subs(T, energie_cinetique)

derivRSDCA_os_cortical = lambda energie_cinetique : derivRSDCA_subs(
    densite              =  1,             #g/cm**3
    excitation           = 106.4*10**(-6), #MeV
    densite_electronique = 0.96*nombre_avogadro).subs(T, energie_cinetique)

#changement de sympy à une expression plus rapide à évaluer
derivRSDCA_eau = lambdify(T, derivRSDCA_eau(T))
derivRSDCA_os_cortical = lambdify(T, derivRSDCA_os_cortical(T))

#Calcul des intégrales pour l'eau et l'os cortical et affichage
RSDCA_eau, nombre_eau = integral_trapese(
    fonction   = derivRSDCA_eau,
    resolution = 10**-9,  #cm
    debut      = 3,       #MeV
    fin        = 220)     #MeV
print("RSDCA eau : ",RSDCA_eau,"cm avec {} trapezes".format(nombre_eau))

RSDCA_os_cortical, nombre_os_cortical = integral_trapese(
    fonction   = derivRSDCA_os_cortical,
    resolution = 10**-9, # cm
    debut      = 3,      # MeV
    fin        = 220)    # MeV
print("RSDCA os cortical : ", RSDCA_os_cortical,"cm avec {} trapezes".format(nombre_os_cortical))


-0.159154943091895*\beta**2*\rho/(c**2*m_e*n_e*r_e**2*(2*\beta**2 - log(4*\beta**2*\gamma**2*c**4*m_e**2*m_p**2*(\gamma**2 - 1)/(I_x**2*(2*\gamma*m_e*m_p + m_e**2 + m_p**2)))))


Résultat Eau :  -4.89161596174964e+46 - 8.25750950991821e+25*I  Résultat Os Cortical :  -2.6441167360795e+46 - 4.46351865397959e+25*I


AttributeError: 'Float' object has no attribute 'rint'

In [8]:
from IPython.display import Latex

##7

On peut simplement utiliser le code de la section précédente pour calculer la différentielle analytique avec sympy. La différentielle est alors:
$\frac{5.25478181043431 \cdot 10^{-27} c^{2} m_{e} n_{e} r_{e}^{2}}{T^{2} \rho \left(1.19570812525521 \cdot 10^{27} T - c^{2}\right)^{8} \left(m_{e}^{2} \sqrt{\frac{1}{c^{2}} \\ \left(- 1.19570812525521 \cdot 10^{27} T + c^{2}\right)} + 2 m_{e} m_{p} + m_{p}^{2} \sqrt{\frac{1}{c^{2}} \left(- 1.19570812525521 \cdot 10^{27} T + c^{2}\right)}\right)}$ $ \left(- 2.39141625051042 \cdot 10^{27} T \left(1.19570812525521 \cdot 10^{27} T - c^{2}\right)^{8} \left(m_{e}^{2} \sqrt{\frac{1}{c^{2}} \left(- 1.19570812525521 \cdot 10^{27} T + c^{2}\right)} + 2 m_{e} m_{p} + m_{p}^{2} \sqrt{\frac{1}{c^{2}} \left(- 1.19570812525521 \cdot 10^{27} T + c^{2}\right)}\right) - 2.09081125 \cdot 10^{-28} c^{2} \left(1.0 T - 8.363245 \cdot 10^{-28} c^{2}\right) \left(- 6.83810133890052 \cdot 10^{81} T m_{e} m_{p} \left(- 1.19570812525521 \cdot 10^{27} T + c^{2}\right)^{6} - 5.71887168320531 \cdot 10^{54} \left(1.19570812525521 \cdot 10^{27} T - c^{2}\right)^{7} \left(m_{e}^{2} \sqrt{\frac{1}{c^{2}} \left(- 1.19570812525521 \cdot 10^{27} T + c^{2}\right)} + 2 m_{e} m_{p} + m_{p}^{2} \sqrt{\frac{1}{c^{2}} \left(- 1.19570812525521 \cdot 10^{27} T + c^{2}\right)}\right) + \left(1.19570812525521 \cdot 10^{27} T - c^{2}\right)^{6} \left(6.83810133890052 \cdot 10^{81} T + 5.71887168320531 \cdot 10^{54} c^{2}\right) \left(m_{e}^{2} \sqrt{\frac{1}{c^{2}} \left(- 1.19570812525521 \cdot 10^{27} T + c^{2}\right)} + 2 m_{e} m_{p} + m_{p}^{2} \sqrt{\frac{1}{c^{2}} \left(- 1.19570812525521 \cdot 10^{27} T + c^{2}\right)}\right)\right) + \left(1.19570812525521 \cdot 10^{27} T - c^{2}\right)^{8} \left(2.39141625051042 \cdot 10^{27} T - c^{2} \left(\log{\left (\frac{T^{2} c^{4} m_{e}^{2} m_{p}^{2} \sqrt{\frac{1}{c^{2}} \left(- 1.19570812525521 \cdot 10^{27} T + c^{2}\right)}}{I_{x}^{2} \left(1.19570812525521 \cdot 10^{27} T - c^{2}\right)^{2} \left(m_{e}^{2} \sqrt{\frac{1}{c^{2}} \left(- 1.19570812525521 \cdot 10^{27} T + c^{2}\right)} + 2 m_{e} m_{p} + m_{p}^{2} \sqrt{\frac{1}{c^{2}} \left(- 1.19570812525521 \cdot 10^{27} T + c^{2}\right)}\right)} \right )} + 126.083366549431\right)\right) \left(m_{e}^{2} \sqrt{\frac{1}{c^{2}} \left(- 1.19570812525521 \cdot 10^{27} T + c^{2}\right)} + 2 m_{e} m_{p} + m_{p}^{2} \sqrt{\frac{1}{c^{2}} \left(- 1.19570812525521 \cdot 10^{27} T + c^{2}\right)}\right)\right)$
